In [1]:
from google.colab import drive
drive.mount('/googledrive')

Mounted at /googledrive


In [3]:
!ls /googledrive/MyDrive/GSSOC24/celeb/

Celeb_Face_Recognition.ipynb  train  val


In [19]:
import os
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, accuracy_score


In [20]:
train_dir = '/googledrive/MyDrive/GSSOC24/celeb/train'
val_dir = '/googledrive/MyDrive/GSSOC24/celeb/val'


In [21]:
def load_data(data_dir):
    images = []
    labels = []
    for label in os.listdir(data_dir):
        if label not in ['madonna', 'mindy_kaling', 'jerry_seinfeld', 'elton_john', 'ben_afflek']:
            continue
        for file in os.listdir(os.path.join(data_dir, label)):
            img_path = os.path.join(data_dir, label, file)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (128, 128))
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)


In [22]:
X_train, y_train = load_data(train_dir)
X_val, y_val = load_data(val_dir)


In [23]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)


In [24]:
X_train = X_train / 255.0
X_val = X_val / 255.0


In [25]:
y_train = to_categorical(y_train, num_classes=len(le.classes_))
y_val = to_categorical(y_val, num_classes=len(le.classes_))


In [26]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(le.classes_), activation='softmax')
])


In [27]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [36]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
3/3 [==============================] - 5s 2s/step - loss: 0.0295 - accuracy: 0.9892 - val_loss: 2.9298 - val_accuracy: 0.6000
Epoch 2/10
3/3 [==============================] - 3s 1s/step - loss: 0.0869 - accuracy: 0.9677 - val_loss: 2.7866 - val_accuracy: 0.5200
Epoch 3/10
3/3 [==============================] - 3s 1s/step - loss: 0.0289 - accuracy: 1.0000 - val_loss: 2.7979 - val_accuracy: 0.5600
Epoch 4/10
3/3 [==============================] - 3s 1s/step - loss: 0.0208 - accuracy: 1.0000 - val_loss: 2.7567 - val_accuracy: 0.6000
Epoch 5/10
3/3 [==============================] - 5s 2s/step - loss: 0.0229 - accuracy: 1.0000 - val_loss: 2.7142 - val_accuracy: 0.6800
Epoch 6/10
3/3 [==============================] - 4s 1s/step - loss: 0.0273 - accuracy: 0.9892 - val_loss: 2.8622 - val_accuracy: 0.6400
Epoch 7/10
3/3 [==============================] - 3s 1s/step - loss: 0.0118 - accuracy: 1.0000 - val_loss: 3.2112 - val_accuracy: 0.6400
Epoch 8/10
3/3 [=========================

In [37]:
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {val_accuracy:.2f}')


1/1 [==============================] - 0s 419ms/step - loss: 2.7513 - accuracy: 0.6400
Validation Accuracy: 0.64


In [38]:
predictions = model.predict(X_val)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_val, axis=1)


1/1 [==============================] - 0s 274ms/step


In [39]:
print(classification_report(true_classes, predicted_classes, target_names=le.classes_))


                precision    recall  f1-score   support

    ben_afflek       0.50      0.60      0.55         5
    elton_john       0.60      0.60      0.60         5
jerry_seinfeld       0.50      0.20      0.29         5
       madonna       0.57      0.80      0.67         5
  mindy_kaling       1.00      1.00      1.00         5

      accuracy                           0.64        25
     macro avg       0.63      0.64      0.62        25
  weighted avg       0.63      0.64      0.62        25



In [40]:
def display_images(images, true_labels, pred_labels, label_encoder, num_images=25):
    plt.figure(figsize=(20, 20))
    for i in range(num_images):
        plt.subplot(5, 5, i + 1)
        plt.imshow(images[i])
        plt.title(f"True: {label_encoder.inverse_transform([true_labels[i]])[0]}\nPred: {label_encoder.inverse_transform([pred_labels[i]])[0]}")
        plt.axis('off')
    plt.show()

# Display images
display_images(X_val, true_classes, predicted_classes, le)


Output hidden; open in https://colab.research.google.com to view.

In [41]:
model.save('/googledrive/MyDrive/GSSOC24/celeb/celebrity_recognition_model.h5')
print("Model saved successfully.")


Model saved successfully.


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [43]:
from tensorflow.keras.models import load_model

loaded_model = load_model('/googledrive/MyDrive/GSSOC24/celeb/celebrity_recognition_model.h5')
print("Model loaded successfully.")


Model loaded successfully.


In [46]:
def predict_and_display_all(val_dir, model, label_encoder):
    for label in os.listdir(val_dir):
        if label not in ['madonna', 'mindy_kaling', 'jerry_seinfeld', 'elton_john', 'ben_afflek']:
            continue
        class_dir = os.path.join(val_dir, label)
        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)
            img = cv2.imread(img_path)
            if img is not None:
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img_resized = cv2.resize(img_rgb, (128, 128))
                img_normalized = img_resized / 255.0
                img_expanded = np.expand_dims(img_normalized, axis=0)

                prediction = model.predict(img_expanded)
                predicted_class = np.argmax(prediction, axis=1)
                predicted_label = label_encoder.inverse_transform(predicted_class)[0]

                plt.imshow(img_rgb)
                plt.title(f"Predicted: {predicted_label}\nActual: {label}")
                plt.axis('off')
                plt.show()

# Use the function to predict and display all validation images
predict_and_display_all(val_dir, loaded_model, le)


Output hidden; open in https://colab.research.google.com to view.